# *btest* initial steps

step-by-step

In [1]:
import argparse
import csv
import itertools
import logging
import os
import sys
import shutil
import time
import math
import random
from scipy.stats import pearsonr
import numpy as np
import pandas as pd
import math
from scipy.stats import spearmanr
from btest import config
from btest import utils
from btest import blockplot
from scipy.stats import t

In [2]:
method = 'spearman'
min_var = 0.0
fdr =0.1

In [3]:
dataX , dataY, featuresX, featuresY  = utils.readData('../demo/X_dataset.txt', '../demo/Y_dataset.txt', min_var=min_var)

Dataset X dimension  after cleaning:  (100, 50)
Dataset Y dimension  after cleaning:  (100, 50)


In [4]:
%%time
within_X = utils.btest_corr(dataX, featuresX, method=method, fdr=fdr, Type='withinX')
within_Y  = utils.btest_corr(dataY, featuresY, method=method, fdr=fdr, Type='withinY')
dataAll = np.concatenate((dataX, dataY), axis=0)
X_Y = utils.btest_corr(dataAll, featuresX, featuresY, method=method, fdr=fdr, Type='X_Y')

CPU times: user 7.8 s, sys: 47.1 ms, total: 7.85 s
Wall time: 7.98 s


In [5]:
X_Y.head()

,Feature_1,Feature_2,pval,Correlation,Not_NAs,P_adusted,bh_fdr_threshold,Type
165,X1,Y66,0.021416,0.327972,49,212.013466,0.65493,X_Y
174,X1,Y75,0.033062,0.305052,49,163.658486,0.65493,X_Y
2747,X27,Y74,0.036443,0.296654,50,120.262541,0.65493,X_Y
173,X1,Y74,0.040235,0.294122,49,99.582635,0.65493,X_Y
1526,X15,Y41,0.042550,0.287992,50,84.249800,0.65493,X_Y


In [6]:
X_Y[(X_Y['Feature_1']=='X0')&(X_Y['Feature_2']=='Y66')]

,Feature_1,Feature_2,pval,Correlation,Not_NAs,P_adusted,bh_fdr_threshold,Type
66,X0,Y66,0.119186,0.223218,50,5.021027,0.65493,X_Y


In [7]:
%%time
X_Y_3 = utils.btest_corr_3(dataAll, featuresX, featuresY, method=method, fdr=fdr, Type='X_Y')

CPU times: user 115 ms, sys: 4.26 ms, total: 120 ms
Wall time: 120 ms


In [8]:
X_Y_3.head()

,Feature_1,Feature_2,complete_obs,Correlation,t_statistic,pval,P_adusted,bh_fdr_threshold,Type
0,X0,X1,49,0.909559,15.004829,0.0,0.0,0.016883,X_X
1,X0,X2,50,0.909350,15.143494,0.0,0.0,0.016883,X_X
2,X0,X3,50,0.932087,17.827314,0.0,0.0,0.016883,X_X
3,X0,X4,49,0.931758,17.593340,0.0,0.0,0.016883,X_X
4,X0,X5,50,0.934547,18.195678,0.0,0.0,0.016883,X_X


In [9]:
X_Y_3[(X_Y_3['Feature_1']=='X0')&(X_Y_3['Feature_2']=='Y66')]

,Feature_1,Feature_2,complete_obs,Correlation,t_statistic,pval,P_adusted,bh_fdr_threshold,Type
165,X0,Y66,50,0.223218,1.586529,0.119186,0.588975,0.016883,X_Y


In [11]:
X_Y_3.loc[:,'Type'].value_counts()

X_Y    10000
X_X     4950
Y_Y     4950
Name: Type, dtype: int64

In [ ]:
X_Y[(X_Y['Feature_1']=='X0')&(X_Y['Feature_2']=='Y66')]

In [ ]:
X_Y_3.loc[:,'Type'] = X_Y_3.Feature_1.str[-1]+X_Y_3.Feature_2.str[-2:]

In [ ]:
X_Y_3.head()

In [ ]:
dataAll = dataX
features = featuresX
features_y = None
Type = 'withinX'
iRow = list(range(0, len(features)))
if Type == 'X_Y':
    iCol = list(range(len(features), len(features)+len(features_y)-1))
    features_y = features + features_y
else:
    features_y = features
    iCol = list(range(len(features)))
tests = []

# creating the complete dataset
dataAll2 = pd.DataFrame(dataAll.T, columns=features_y)
cr = dataAll2.corr(method=method)

# calculating t-statistics, based on the correlations
t_stat = (cr*(dataAll2.shape[0]-2)**.5)/(1-cr**2)**.5

# calculating p-values based on the t-statistics
pv = 2 * (1 - t.cdf(abs(t_stat), df=dataAll2.shape[0]-2))
#rho_long_format = cr.stack().reset_index().rename(columns={'level_0':'Feature_1','level_1':'Feature_2', 0:'Correlation'})
#pval_long_format = pv.stack().reset_index().rename(columns={'level_0':'Feature_1','level_1':'Feature_2', 0:'pval'})
#results = pd.concat([pval_long_format, rho_long_format["Correlation"]], axis=1)


In [ ]:
cr = cr.to_numpy()
#pv = pv.to_numpy()

In [ ]:
for i in iRow:
    for j in iCol:
        if i<=j:
            #print(i, j)
            #print(dataAll[i])
            X = dataAll[i]
            Y = dataAll[j]
            nas = np.logical_or(X != X, Y != Y)
            not_na = sum(~nas)
            #X = Y[~nas]
            #Y = Y[~nas]
            #new_X, new_Y = remove_pairs_with_a_missing(X, Y)
            correlation = cr[i, j]
            pval =  pv[i, j]
            tests.append([features[i], features_y[j], pval, correlation, not_na])
results = pd.DataFrame(tests, columns=['Feature_1','Feature_2','pval', 'Correlation', 'Not_NAs'])

p_adust, p_threshold = bh(results["pval"].values, fdr)

results["P_adusted"] = p_adust
results["bh_fdr_threshold"] = p_threshold
results['Type'] = Type
results = results.sort_values(['pval', 'Correlation'],
                              ascending=[True, False])

In [ ]:
# feature names for the complete dataset
featuresAll = featuresX.copy()
featuresAll.extend(featuresY)

In [ ]:
results

In [ ]:
# creating the complete dataset
dataAll_df = pd.DataFrame(dataAll.T, columns=featuresAll)

In [ ]:
%%time
# correlation
cr = dataAll_df.corr(method='pearson')

# calculaing t-statistics, based on the correlations
t_stat = (cr*(dataAll_df.shape[0]-2)**.5)/(1-cr**2)**.5

# calculating p-values based on the t-statistics
pv = 2 * (1 - t.cdf(abs(t_stat), df=dataAll_df.shape[0]-2))

In [ ]:
cr

In [ ]:
%%time
# correlation
cr = dataAll_df.corr(method='pearson')

# calculaing t-statistics, based on the correlations
t_stat = (cr*(dataAll_df.shape[0]-2)**.5)/(1-cr**2)**.5

# calculating p-values based on the t-statistics
pv = 2 * (1 - t.cdf(abs(t_stat), df=dataAll_df.shape[0]-2))

In [ ]:
X_Y

In [ ]:
within_X

In [ ]:
within_Y

In [ ]:
X_Y.pval

In [ ]:
outputpath = "/Users/rah/Documents/omicsEye/btest/demo/btest_output"
os.makedirs(outputpath, exist_ok=True)
X_Y.to_csv(outputpath + '/X_Y.tsv', sep="\t")
#rho_X = rho.iloc[0:500, 0:500]
#rho_X.to_csv(outputpath + '/simtable_X.tsv', sep="\t")
#rho_X_Y.to_csv(outputpath + '/simtable.tsv', sep="\t")
within_X.to_csv(outputpath + '/within_X.tsv', sep="\t")
within_Y.to_csv(outputpath + '/within_Y.tsv', sep="\t")

In [ ]:
#associations = blockplot.load_associations(path = outputpath + '/X_Y.tsv')
associations = blockplot.load_associations('/Users/rah/Dropbox/Ali-Docs/Research_docs/Projects/VastBiome/analysis/btest_bgc_mgx_replaceNA_FALSE_cov_25_measure_rpkg_mbx_pair_spearman_new/X_Y.tsv')

In [ ]:
outputpath

In [ ]:
associations

In [ ]:
#simtable = blockplot.load_order_table(outputpath + '/simtable.tsv', associations)
simtable = blockplot.load_order_table('/Users/rah/Dropbox/Ali-Docs/Research_docs/Projects/VastBiome/analysis/btest_bgc_mgx_replaceNA_FALSE_cov_25_measure_rpkg_mbx_pair_spearman_new//simtable.tsv', associations)

In [ ]:
X_Y

In [ ]:
blockplot.plot(
    simtable,
    associations,
    cmap="RdBu_r",
    mask=False,
    axlabels=["",""],
    outfile="/Users/rah/Documents/omicsEye/btest/demo/btest_output/blockplot.pdf",
    similarity="Spearman"
)